In [13]:
import pandas as pd
df = pd.read_csv('diabetes.csv')
df

,Age,Gender,Polyuria,Polydipsia,sudden weight loss,weakness,Polyphagia,Genital thrush,visual blurring,Itching,Irritability,delayed healing,partial paresis,muscle stiffness,Alopecia,Obesity,class
0,40,Male,No,Yes,No,Yes,No,No,No,Yes,No,Yes,No,Yes,Yes,Yes,Positive
1,58,Male,No,No,No,Yes,No,No,Yes,No,No,No,Yes,No,Yes,No,Positive
2,41,Male,Yes,No,No,Yes,Yes,No,No,Yes,No,Yes,No,Yes,Yes,No,Positive
3,45,Male,No,No,Yes,Yes,Yes,Yes,No,Yes,No,Yes,No,No,No,No,Positive
4,60,Male,Yes,Yes,Yes,Yes,Yes,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
515,39,Female,Yes,Yes,Yes,No,Yes,No,No,Yes,No,Yes,Yes,No,No,No,Positive
516,48,Female,Yes,Yes,Yes,Yes,Yes,No,No,Yes,Yes,Yes,Yes,No,No,No,Positive
517,58,Female,Yes,Yes,Yes,Yes,Yes,No,Yes,No,No,No,Yes,Yes,No,Yes,Positive
518,32,Female,No,No,No,Yes,No,No,Yes,Yes,No,Yes,No,No,Yes,No,Negative


# define col

In [14]:
x_col = df.drop('class',axis=1).columns.tolist()
y_col = 'class'
x_col,y_col

(['Age',
  'Gender',
  'Polyuria',
  'Polydipsia',
  'sudden weight loss',
  'weakness',
  'Polyphagia',
  'Genital thrush',
  'visual blurring',
  'Itching',
  'Irritability',
  'delayed healing',
  'partial paresis',
  'muscle stiffness',
  'Alopecia',
  'Obesity'],
 'class')

# LabelEncoder

In [15]:
from sklearn import preprocessing
le_dict = {}
for col in df.columns:
    le_dict[col] = preprocessing.LabelEncoder()
    le_dict[col].fit(df[col])
    df[col] = le_dict[col].transform(df[col])
df

,Age,Gender,Polyuria,Polydipsia,sudden weight loss,weakness,Polyphagia,Genital thrush,visual blurring,Itching,Irritability,delayed healing,partial paresis,muscle stiffness,Alopecia,Obesity,class
0,16,1,0,1,0,1,0,0,0,1,0,1,0,1,1,1,1
1,34,1,0,0,0,1,0,0,1,0,0,0,1,0,1,0,1
2,17,1,1,0,0,1,1,0,0,1,0,1,0,1,1,0,1
3,21,1,0,0,1,1,1,1,0,1,0,1,0,0,0,0,1
4,36,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
515,15,0,1,1,1,0,1,0,0,1,0,1,1,0,0,0,1
516,24,0,1,1,1,1,1,0,0,1,1,1,1,0,0,0,1
517,34,0,1,1,1,1,1,0,1,0,0,0,1,1,0,1,1
518,8,0,0,0,0,1,0,0,1,1,0,1,0,0,1,0,0


# 特徵擷取

In [17]:
X = df[x_col]
y = df[y_col]

# Tree-based feature selection

In [19]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
clf = ExtraTreesClassifier(n_estimators=50)
clf = clf.fit(X, y)
print(clf.feature_importances_ )
model = SelectFromModel(clf, prefit=True)
X_new = model.transform(X)
X.shape,X_new.shape   

[0.04868436 0.10372852 0.25184114 0.16894573 0.0604495  0.01841551
 0.03746273 0.02151096 0.03751522 0.03517196 0.04210686 0.03674353
 0.04613692 0.02652196 0.04430293 0.02046217]


((520, 16), (520, 3))

# Univariate feature selectio

In [20]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
X_new = SelectKBest(chi2, k=2).fit_transform(X, y)
X.shape,X_new.shape

((520, 16), (520, 2))

# 學習演算法

In [23]:
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(X, y)
clf.score(X,y)

0.9826923076923076

In [25]:
from xgboost import XGBClassifier
clf = XGBClassifier()
clf.fit(X,y)
clf.score(X,y)

[06:50:31] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


c:\Users\ricky\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


1.0

# 參數最佳化

In [29]:
import optuna

def objective(trial):
    x1 = trial.suggest_float("C", 1, 100)
    x2 = trial.suggest_float("gamma", 0.01, 1)
    clf = make_pipeline(StandardScaler(), SVC(C = x1,gamma=x2))
    clf.fit(X, y)
    return clf.score(X,y)
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials = 50)

[I 2022-06-28 06:55:27,953] A new study created in memory with name: no-name-8504d9b8-04de-4d02-99dc-893455158ca3
[I 2022-06-28 06:55:27,971] Trial 0 finished with value: 1.0 and parameters: {'C': 44.555240173337594, 'gamma': 0.3279020921385849}. Best is trial 0 with value: 1.0.
[I 2022-06-28 06:55:27,985] Trial 1 finished with value: 1.0 and parameters: {'C': 39.09845985181177, 'gamma': 0.7660052263325916}. Best is trial 0 with value: 1.0.
[I 2022-06-28 06:55:28,014] Trial 2 finished with value: 0.9980769230769231 and parameters: {'C': 54.22872061930576, 'gamma': 0.24677530991452737}. Best is trial 0 with value: 1.0.
[I 2022-06-28 06:55:28,030] Trial 3 finished with value: 0.9942307692307693 and parameters: {'C': 19.313274639407677, 'gamma': 0.3073238083461566}. Best is trial 0 with value: 1.0.
[I 2022-06-28 06:55:28,045] Trial 4 finished with value: 1.0 and parameters: {'C': 66.07760714043691, 'gamma': 0.6976551928915334}. Best is trial 0 with value: 1.0.
[I 2022-06-28 06:55:28,064] 

In [34]:
study.best_params['C']
study.best_params['gamma']

0.3279020921385849

# 集成學習

In [35]:
import numpy as np
from sklearn.ensemble import VotingClassifier
clf1 = SVC(C = study.best_params['C'],gamma =study.best_params['gamma'])
clf2 = XGBClassifier()
eclf1 = VotingClassifier(estimators=[('SVC', clf1), ('XGB', clf2)], voting='hard')
eclf1 = eclf1.fit(X, y)
eclf1.score(X,y)

[06:58:49] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


c:\Users\ricky\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


1.0